here only train n validation is done


In [1]:
import os
import pandas as pd
import shutil
from sklearn.model_selection import train_test_split
from PIL import Image

Cloning the Yolov5 Repository for model traning 

In [2]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...


Spliting the Images into Train and Validation for model training and evaluation

In [3]:
# Paths
images_dir = 'Data/license_plates_detection_train'
test_img_dir='Data/test'
# Read annotations
df = pd.read_csv('Data/Licplatesdetection_train.csv')

# Split into train and validation (80% train, 20% val)
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [4]:
# YOLOv5 directory structure
yolov5_data_dir = 'yolov5/custom_data'
directories = {
    "train_images": os.path.join(yolov5_data_dir, 'train','images'),
    "val_images": os.path.join(yolov5_data_dir,'val','images'),
    "train_labels": os.path.join(yolov5_data_dir, 'train','labels'),
    "val_labels": os.path.join(yolov5_data_dir,  'val','labels'),

}

# Create directories if not exist
for path in directories.values():
    os.makedirs(path, exist_ok=True)



'yolov5/custom_data/test'

Copying the test images to the desired directory for testing. 

In [ ]:
shutil.copytree(test_img_dir,yolov5_data_dir+'/test')

Saving the Train and validation dataset csv file with their Bounded Box Annotation

In [5]:
train_df.to_csv(os.path.join('yolov5/custom_data/train','Licplatesdetection_train.csv'), index=False)
val_df.to_csv(os.path.join('yolov5/custom_data/val',"Licplatesdetection_val.csv"), index=False)

Converting the Bounded Box annotations into labels.txt for model training

In [6]:
# Function to convert and save labels in YOLO format
def convert_and_save(df_subset, images_output_dir, labels_output_dir):
    for _, row in df_subset.iterrows():
        img_id = row['img_id']
        img_path = os.path.join(images_dir, img_id)
        if not os.path.exists(img_path):
            print(f"Image {img_id} not found.")
            continue

        # Copy image to the respective directory
        shutil.copy(img_path, images_output_dir)

        # Open image to get dimensions
        with Image.open(img_path) as img:
            width, height = img.size

        # Convert bounding box to YOLO format
        x_center = ((row['xmin'] + row['xmax']) / 2) / width
        y_center = ((row['ymin'] + row['ymax']) / 2) / height
        bbox_width = (row['xmax'] - row['xmin']) / width
        bbox_height = (row['ymax'] - row['ymin']) / height

        # YOLO format: class x_center y_center width height
        # Assuming class 0 for license plate
        yolo_format = f"0 {x_center} {y_center} {bbox_width} {bbox_height}\n"

        # Save label file
        label_filename = os.path.splitext(img_id)[0] + '.txt'
        label_path = os.path.join(labels_output_dir, label_filename)
        with open(label_path, 'w') as label_file:
            label_file.write(yolo_format)


# Convert and save train and validation data
convert_and_save(train_df, directories["train_images"], directories["train_labels"])
convert_and_save(val_df, directories["val_images"], directories["val_labels"])

checking if the torch and CUDA installed and avialable for model training

In [7]:

import torch
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))


Setup complete. Using torch 2.4.1 _CudaDeviceProperties(name='NVIDIA GeForce RTX 3050 Laptop GPU', major=8, minor=6, total_memory=4095MB, multi_processor_count=16)


Installing the required libraries 

In [8]:
%cd yolov5
!pip install -r requirements.txt
!pip install torch torchvision torchaudio
!pip install pandas scikit-learn pillow

c:\Users\ARISH\Desktop\2nd_final\yolov5


Creating data.yaml file which contains the location of train and val images also the number of class to be identified

In [9]:

!echo. > data.yaml
# Modify data.yaml to point to the correct paths

# data.yaml
# train: ./custom_data/train/images
# val: ./custom_data/val/images

# nc: 1
# names: ['license_plate']

Customizing the yolov5 model for your specific use case but changing the number of class to 1 to detect  only  one class i.e license_plate

In [13]:
!copy models\yolov5s.yaml models\custom_yolov5s.yaml

        1 file(s) copied.



After copying change the code as below

In [11]:
# custom_yolov5s.yaml
# parameters
# nc: 1  # number of classes
# depth_multiple: 0.33
# width_multiple: 0.50


Training the model 

In [14]:
!python train.py --img 640 --batch 16 --epochs 20 --data data.yaml --cfg models/custom_yolov5s.yaml --weights yolov5s.pt --name license_plate_detector

train: weights=yolov5s.pt, cfg=models/custom_yolov5s.yaml, data=data.yaml, hyp=data\hyps\hyp.scratch-low.yaml, epochs=20, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data\hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs\train, name=license_plate_detector, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: up to date with https://github.com/ultralytics/yolov5 
YOLOv5  v7.0-366-gf7322921 Python-3.10.14 torch-2.4.1 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=